In [1]:
from flask import Flask, request, jsonify
import pandas as pd
import json
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
app = Flask(__name__)

In [3]:
class Tourism:
    def __init__(self):
        self.tourism_place = None
        self.tfidf_matrix_tourism = None
        self.similarity_tensor = None
        self.load_data("C:/Users/LENOVO/Downloads/Wisata_Data.csv")
        self.filter_data()
        self.tfidf_matrix_tourism, self.tfidf = self.TF_IDF()
        self.calculate_similarity(self.tfidf_matrix_tourism)
        
    def load_data(self, filename):
        self.tourism_place = pd.read_csv(filename)
    
    def filter_data(self):
        self.tourism_place = self.tourism_place.drop(columns="Unnamed: 0")
        self.tourism_place = self.tourism_place[["place_id", "name", "types", "formatted_address", "user_ratings_total", "rating"]]
        self.tourism_place = self.tourism_place.drop_duplicates(subset="place_id")
        self.tourism_place = self.tourism_place.reset_index(drop=True)
        
    def preprocess_text(self):
        combined_text = self.tourism_place['name'] + ' ' + self.tourism_place['types'] + ' ' + self.tourism_place['formatted_address']
        return combined_text
    
    def TF_IDF(self):
        tfidf = TfidfVectorizer()
        combined_text = self.preprocess_text()
        tfidf_matrix = tfidf.fit_transform(combined_text)
        return tfidf_matrix, tfidf
    
    def calculate_similarity(self, tfidf_matrix):
        similarity_matrix = cosine_similarity(tfidf_matrix)
        self.similarity_tensor = tf.convert_to_tensor(similarity_matrix)
    
    def recommend(self, input_text, k):
        tfidf_matrix, tfidf = self.TF_IDF()
        
        tfidf_matrix_input = tfidf.transform([input_text])
        similarity_scores = cosine_similarity(tfidf_matrix_input, tfidf_matrix)
        similarity_scores = similarity_scores.flatten()
        
        top_k_values, top_k_indices = tf.nn.top_k(similarity_scores, k=k)
        top_k_places = self.tourism_place.loc[top_k_indices]
        
        return top_k_places

class Restaurant:
    def __init__(self):
        self.restaurant_place = None
        self.tfidf_matrix_restaurant = None
        self.similarity_tensor = None
        self.load_data("C:/Users/LENOVO/Downloads/Restaurant_Data.csv")
        self.filter_data()
        self.tfidf_matrix_restaurant, self.tfidf = self.TF_IDF()
        self.calculate_similarity(self.tfidf_matrix_restaurant)
        
    def load_data(self, filename):
        self.restaurant_place = pd.read_csv(filename)
    
    def filter_data(self):
        self.restaurant_place = self.restaurant_place.drop(columns="Unnamed: 0")
        self.restaurant_place = self.restaurant_place[["place_id", "name", "types", "formatted_address", "user_ratings_total", "rating"]]
        self.restaurant_place = self.restaurant_place.drop_duplicates(subset="place_id")
        self.restaurant_place = self.restaurant_place.reset_index(drop=True)
        
    def preprocess_text(self):
        combined_text = self.restaurant_place['name'] + ' ' + self.restaurant_place['types'] + ' ' + self.restaurant_place['formatted_address']
        return combined_text
    
    def TF_IDF(self):
        tfidf = TfidfVectorizer()
        combined_text = self.preprocess_text()
        tfidf_matrix = tfidf.fit_transform(combined_text)
        return tfidf_matrix, tfidf
    
    def calculate_similarity(self, tfidf_matrix):
        similarity_matrix = cosine_similarity(tfidf_matrix)
        self.similarity_tensor = tf.convert_to_tensor(similarity_matrix)
    
    def recommend(self, input_text, k):
        tfidf_matrix, tfidf = self.TF_IDF()
        
        tfidf_matrix_input = tfidf.transform([input_text])
        similarity_scores = cosine_similarity(tfidf_matrix_input, tfidf_matrix)
        similarity_scores = similarity_scores.flatten()
        
        top_k_values, top_k_indices = tf.nn.top_k(similarity_scores, k=k)
        top_k_places = self.restaurant_place.loc[top_k_indices]
        
        return top_k_places

class Hotel:
    def __init__(self):
        self.hotel_place = None
        self.tfidf_matrix_hotel = None
        self.similarity_tensor = None
        self.load_data("C:/Users/LENOVO/Downloads/Hotel_Data.csv")
        self.filter_data()
        self.tfidf_matrix_hotel, self.tfidf = self.TF_IDF()
        self.calculate_similarity(self.tfidf_matrix_hotel)
        
    def load_data(self, filename):
        self.hotel_place = pd.read_csv(filename)
    
    def filter_data(self):
        self.hotel_place = self.hotel_place.drop(columns="Unnamed: 0")
        self.hotel_place = self.hotel_place[["place_id", "name", "types", "formatted_address", "user_ratings_total", "rating"]]
        self.hotel_place = self.hotel_place.drop_duplicates(subset="place_id")
        self.hotel_place = self.hotel_place.reset_index(drop=True)
        
    def preprocess_text(self):
        combined_text = self.hotel_place['name'] + ' ' + self.hotel_place['types'] + ' ' + self.hotel_place['formatted_address']
        return combined_text
    
    def TF_IDF(self):
        tfidf = TfidfVectorizer()
        combined_text = self.preprocess_text()
        tfidf_matrix = tfidf.fit_transform(combined_text)
        return tfidf_matrix, tfidf
    
    def calculate_similarity(self, tfidf_matrix):
        similarity_matrix = cosine_similarity(tfidf_matrix)
        self.similarity_tensor = tf.convert_to_tensor(similarity_matrix)
    
    def recommend(self, input_text, k):
        tfidf_matrix, tfidf = self.TF_IDF()
        
        tfidf_matrix_input = tfidf.transform([input_text])
        similarity_scores = cosine_similarity(tfidf_matrix_input, tfidf_matrix)
        similarity_scores = similarity_scores.flatten()
        
        top_k_values, top_k_indices = tf.nn.top_k(similarity_scores, k=k)
        top_k_places = self.hotel_place.loc[top_k_indices]
        
        return top_k_places

In [4]:
tourism = Tourism()
restaurant = Restaurant()
hotel = Hotel()

In [5]:
@app.route('/recommend-tourism', methods=['POST'])
def recommend_tourism_places():
    data = request.json
    input_text = data['input_text']
    k = data['k']

    recommended_places = tourism.recommend(input_text, k)
    recommended_places_json = json.dumps(recommended_places.to_dict(orient='records'), ensure_ascii=False)

    return recommended_places_json

@app.route('/recommend-restaurant', methods=['POST'])
def recommend_restaurant_places():
    data = request.json
    input_text = data['input_text']
    k = data['k']

    recommended_places = restaurant.recommend(input_text, k)
    recommended_places_json = json.dumps(recommended_places.to_dict(orient='records'), ensure_ascii=False)

    return recommended_places_json
    
@app.route('/recommend-hotel', methods=['POST'])
def recommend_hotel_places():
    data = request.json
    input_text = data['input_text']
    k = data['k']

    recommended_places = hotel.recommend(input_text, k)
    recommended_places_json = json.dumps(recommended_places.to_dict(orient='records'), ensure_ascii=False)

    return recommended_places_json

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2023 13:48:48] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 13:48:51] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 13:49:04] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 13:49:32] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 13:49:47] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 14:27:23] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 14:27:39] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 14:28:02] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 14:28:13] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 14:28:23] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 19:05:24] "POST /recommend-tourism HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 19:05:44] "POST /recommend-tourism HTTP/1.1" 200 -
127.